In [98]:
import pandas as pd
import json
import re

In [99]:
with open('../../metadata/youtube-api/videos_list_yutt.json') as f:
    videos_list_yutt = json.loads(f.read())

In [100]:
videos = [v for v in videos_list_yutt if 'Dhammapada' in v['snippet']['title']]
titles = [v['snippet']['title'] for v in videos]

In [101]:
numbers = []
mapping = {}

for i, title in enumerate(titles):
    m = re.findall(r'([0-9]+)', title)
    
    new_numbers = []
    if len(m) == 3:
        new_numbers = [int(x) for x in m]
    elif len(m) == 2:
        new_numbers = list(range(int(m[0]), int(m[1]) + 1))
    elif len(m) == 1:
        new_numbers = [int(m[0])]
    
    numbers += new_numbers
    
    for n in new_numbers:
        mapping[n] = videos[i]

In [102]:
[x for x in range(1, 224) if not x in numbers] # written numbers missing

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [103]:
with open('verses.txt') as f:
    verses = f.read().split('\n---\n')

In [104]:
exceptions = [207, 210, 198, 189, 190, 191, 184, 138, 139, 88, 10, 9, 8, 7, 6, 5, 4, 3, 2]

for verse in verses:
    for num in [int(n) for n in re.findall(r'[0-9]+', verse)]:
        assert (num in exceptions or str(num) in mapping[num]['snippet']['title'])
        
for i, verse in enumerate(verses):
    for num in [int(n) for n in re.findall(r'[0-9]+', verse)]:
        assert (num in exceptions or str(num) in videos[i]['snippet']['title'])

In [105]:
df = pd.read_json('../../metadata/subtitles/top_keywords_from_subtitles.json')
keywords = { row.link: row.keywords for row in df.itertuples() }

In [106]:
def create_desc(verse, video, tags):
    template = """%s

Part of a series on the Dhammapada, a set of 423 verse teachings given by the Buddha, including explanation of the Pali verse, a synopsis of the background story and application of the teaching to our practice.

<b>Original YouTube-Video:</b> %s
<b>For more information on the Dhammapada:</b> http://www.buddhanet.net/e-learning/buddhism/dhammapada.htm
<b>Automatic Keywords:</b> %s
""" % (verse, video, tags)
    return template

In [112]:
items = []

for i in range(len(videos)):
    link = 'https://youtu.be/' + videos[i]['id']
    tags = 'None' if not link in keywords else keywords[link]
    tags = ', '.join(tags.split(',')[:3])
    desc = create_desc(verses[i], link, tags)
    items.append({
        'title': videos[i]['snippet']['title'],
        'description': desc,
        'duration': ''
    })